In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from df import blocks as blocks

In [3]:
p = blocks.create_beam_pipeline()

In [4]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

In [5]:
create = blocks.CreateBlock(values = sentences)

In [6]:
create.operation

<Create(PTransform) label=[Create] at 0x288418370>

In [7]:
create

CreateBlock(block_id=UUID('2bbbc4ed-d3ea-4dc5-b4e5-625e03c314d9'), source_ids=[], target_ids=[], operation=<Create(PTransform) label=[Create] at 0x288418370>, o=None, values=['This framework generates embeddings for each input sentence', 'Sentences are passed as a list of string.', 'The quick brown fox jumps over the lazy dog.'])

In [8]:
embed = blocks.SentenceEmbeddingBlock()

In [9]:
embed.model_name

'all-MiniLM-L6-v2'

In [10]:
embed.operation

<ParDo(PTransform) label=[Map(<lambda at blocks.py:54>)] at 0x177efeeb0>

In [11]:
model = blocks.BlockAssembler.Sequential([create, embed], p=p)

In [12]:
model.compile()

In [13]:
len(model.blocks)

2

In [14]:
model.show_graph()

/Users/xqhu/homebrew/bin/dot


In [15]:
model.block_data(model.blocks[0])

,0
0,This framework generates embeddings for each i...
1,Sentences are passed as a list of string.
2,The quick brown fox jumps over the lazy dog.


In [16]:
model.block_data(model.blocks[1])

,0
0,"[-0.013717398, -0.042851556, -0.01562865, 0.01..."
1,"[0.0564525, 0.05500244, 0.031379532, 0.0339484..."
2,"[0.043933515, 0.05893439, 0.04817844, 0.077548..."


In [17]:
model.blocks[1].json(exclude={"operation", "o"})

'{"block_id": "5964d7fc-f313-4433-8671-ab3c803b7169", "source_ids": ["2bbbc4ed-d3ea-4dc5-b4e5-625e03c314d9"], "target_ids": [], "model_name": "all-MiniLM-L6-v2"}'